### Creazione del Dizionario JSON del Dataset

In [13]:
import os
import json
import re
import pprint

#### Specifiche dei paragrafi

- i paragrafi validi per l’analisi dovranno contenere un numero di token compreso tra 50 e 100 
- i token potranno essere approssimati dividendo il testo sugli spazi, senza ricorrere a una tokenizzazione linguistica
avanzata

In [14]:
# mi muovo nella cartella dove sono contenuti i miei file e ciclo sulle sotto cartelle e sui file per ottenerli tutti (solo quelli già normalizzati)

lista_file = []
for cartella, sottocartelle, files in os.walk("Testi progetto"):
    for file in files:
        if file.endswith("_norm.txt"):
            file_path = cartella+"/"+file
            lista_file.append(file_path)
print(lista_file)

['Testi progetto/Serao Matilde - 1856-1927/serao_gli_amanti_norm.txt', 'Testi progetto/Serao Matilde - 1856-1927/serao_piccole_anime_norm.txt', 'Testi progetto/Serao Matilde - 1856-1927/serao_l_infedele_norm.txt', 'Testi progetto/Serao Matilde - 1856-1927/serao_le_amanti_norm.txt', 'Testi progetto/Serao Matilde - 1856-1927/serao_la_conquista_di_roma_norm.txt', 'Testi progetto/Neera Zuccari Anna Maria - 1846 - 1918/neera_l_amuleto_norm.txt', 'Testi progetto/Neera Zuccari Anna Maria - 1846 - 1918/neera_le_idee_di_una_donna_norm.txt', 'Testi progetto/Neera Zuccari Anna Maria - 1846 - 1918/neera_la_vecchia_casa_norm.txt', 'Testi progetto/Neera Zuccari Anna Maria - 1846 - 1918/neera_rogo_d_amore_norm.txt', 'Testi progetto/Neera Zuccari Anna Maria - 1846 - 1918/neera_il_sogno_norm.txt', 'Testi progetto/Colombi Marchesa, 1840-1920/colombi_tempesta_e_bonaccia_norm.txt', 'Testi progetto/Colombi Marchesa, 1840-1920/colombi_cara_speranza_norm.txt', 'Testi progetto/Colombi Marchesa, 1840-1920/colo

Funzione che prende in input la lista dei token della frase e controlla la lunghezza: se è compresa tra 50 e 100, appende direttamente il paragrafo; se è maggiore di 100, spezza la frase al primo punto presente, controlla la lunghezza, verifica la validità, appende se necessario e poi richiama sé stessa sul resto del paragrafo

In [15]:
paragrafi_scelti = []

def elabora_lista_token(lista_token):

    global paragrafi_scelti
    # se il paragrafo è minore di 50 token, lo scarta
    if len(lista_token)<50:
       return
    # se è compreso tra 50 e 100, lo appende
    elif(len(lista_token)>=50 and len(lista_token)<=100):
        paragrafi_scelti.append(" ".join(lista_token))
        return
    # altrimenti, scorre il paragrafo al primo segno di punteggiatura disponibile
    else:
        contatore = 49
        while not lista_token[contatore].endswith(".") and contatore < len(lista_token)-1:
            contatore +=1
        # se il paragrafo risultante è minore di 100, lo appende
        if contatore <= 100:
            paragrafi_scelti.append(" ".join(lista_token[0:contatore+1]))
        # rilancia sé stessa e riparte
        elabora_lista_token(lista_token[contatore+1:])

In [16]:
def selezione_paragrafi(libro):

    global paragrafi_scelti
    paragrafi_scelti = []
    # Spezzo il testo in paragrafi
    lista_paragrafi = libro.split("\n\n")
    # Scorro i paragrafi nel testo. Per ciascuno, creo una lista di token splittando sugli spazi. 
    for paragrafo in lista_paragrafi:
        lista_token_paragrafo = paragrafo.split(" ")
        # chiamo la funzione che spezza la frase quando necessario
        elabora_lista_token(lista_token_paragrafo)
    return paragrafi_scelti


#### Ciclo la lista di file, per ciascuno lancio la funzione di selezione paragrafi

In [17]:
# Creo un dizionario che conterrà i paragrafi scelti di ogni autrice.
# Assumo (visto che le specifiche non lo precisano) che non mi serva portarmi dietro il nome del libro dal quale viene ciascun capitolo.
dizionario_autrici = {}
for file in lista_file:

    f = open(file)
    libro = f.read()
    # Ottengo la lista di paragrafi scelti del file
    paragrafi = selezione_paragrafi(libro)

    autrice = file.split("/")[1]

    # Scorro il dizionario delle autrici. 
    # Se l'autrice non è presente, creo una chiave a suo nome e inserisco la lista appena estratta come valore
    if autrice not in dizionario_autrici:
        dizionario_autrici[autrice] = {}
        
    # Ottengo il nome dell'autrice e il nome del libro (da usare come chiave)
    nome_file = file.split("/")[2]
    nome_libro = nome_file.replace("_norm.txt","")
    
    dizionario_autrici[autrice][nome_libro] = paragrafi

##### Salvo il dizionario creato in un file json

In [18]:
with open("dizionario_paragrafi.json", "w", encoding = 'utf8') as outfile:
    json.dump(dizionario_autrici, outfile, ensure_ascii=False)
